To check the connectivity of phonon modes on volumes, fit phonon modes with volumes using polynomial functions up to 5, and fit equations of state. Similar to the QHA module in CRYSTAL17. Required files & input parameters:
* Harmonic phonon calculation output by CRYSTAL17  
* The name list of harmonic phonon outputs, stored with temperature range in 'HA_input.dat'  

By Spica. Vir., ICL, May 13 - 22. spica.h.zhou@gmail.com

In [ ]:
import math
import os
import numpy as np

In [ ]:
class Mode:
    """
    mode class - store important information for mode with the same rank.
        rank, int, the rank of the mode object
        npoint, int, number of harmonic frequency calculations
        freq, numpy array, npoint * 1 array of frequencies corresponding to the same mode
        vol, numpy array, npoint * 1 array of volumes of harmonic input
        eigvt, numpy array, npoint * 3 array of corresponding eigenvector
        poly_fit, numpy array, npower * 1 array of fitted polynomials
    """

    def __init__(self, rank=0, npoint=0, freq=[], vol=[],  eigvt=[]):
        self.rank = rank
        self.npoint = npoint
        # Unit: THz
        self.freq = np.array(freq, dtype=float) * 2 * math.pi
        # Unit: A^3
        self.vol = np.array(vol, dtype=float)
        # Unit: A
        self.eigvt = np.array(eigvt, dtype=float)
        self.poly_fit = np.array([], dtype=float)

    def continuity(self, wtout, threshold=0.4):
        """
        Check the continuity of phonon modes (NB. Not sure - re-check needed)
        """
        wtout.write('%-15s%6i\n' % ('FREQUENCY #', self.rank))
        wtout.write('%-15s%15s\n' % ('  VOL (A^3)', 'FREQ (cm^-1)'))
        for i in range(self.npoint):
            wtout.write('%2s%-15.4f%15.4f\n' % ('', self.vol[i], self.freq[i]))
            for j in range(i + 1, self.npoint):
                dot_eigvt = np.dot(self.eigvt[i], self.eigvt[j])
                if dot_eigvt < threshold:
                    wtout.write('%s%i%s\n'% (
                        'Warning: Continuity between the above mode and mode ',
                        j + 1, ' is problematic.'))

        wtout.write('\n')
        
        return self
        
    def poly_fit(self, wtout, power=[2, 3, 4]):
        """
        Fit phonon frequency as the polynomial function of volume
        """
        if max(power) > self.npoint - 1:
            wtout.write('%s\n' % 'Error: Lack referece data for polynomial fit.')   
            return
        
        wtout.write('%-6s%9s%12s%4s%s\n' % ('FREQ #', 'POWER', 'R^2', '',
                                            'COEFF LOW TO HIGH'))
        wtout.write('%6i' % self.rank)
        r_squares = np.array([])
        for i in power:
            if i != power[0]:
                wtout.write('%6i' % '')

            func = np.polynomial.polynomial.Polynomial.fit(self.vol, self.freq, i)
            self.poly_fit = np.append(self.poly_fit, func)
            coeff = func.convert().coef
            ss_res = np.sum((self.freq - func(self.vol))**2)
            ss_tot = np.sum((self.freq - np.mean(self.freq))**2)
            r_square = 1 - ss_res / ss_tot

            wtout.write('%9i%12.6f%4s' % (i, r_square, ''))
            for c in coeff:
                wtout.write('%-10.4e' % c)
                
            wtout.write('\n')
            r_squares = np.append(r_squares, r_square)
        
        r_squares = np.array(r_squares)
        
        return self, r_squares
    

In [ ]:
class ReadHAdata:
    """
    ReadHAdata class - read the outputs of harmonic frequency calculations and input setting
    files. 
    """
    def __init__(self, setfilename):
        if os.path.isfile(setfilename):
            file = open(setfilename, "r")
            self.setdata = file.readlines()
            file.close()
        else:
            print('Error: File not exist.')
            return
        
    def readhaout(self):
        """
        Read the input settings file for harmonic calculation outputs.Return to a name
        list of .out files
        """
        label_f = 'FILES'
        label_e = 'END'
        label_seq = 0

        countline = 0
        while countline < len(self.setdata):
            if label_f in self.setdata[countline]:
                countline += 1
                line_f = countline
                label_seq = 1
            elif label_e in data and lable_seq:
                line_e = countline
                break

            countline += 1
                
        if not label_f or not label_e:
            print('Error: Keyword \'FILES\' or \'END\' missing.')
            return self
        
        self.hafilename = [i.strip() for i in self.setdata[line_f:line_e]]
        
        self.hadata = []
        for ha in self.hafile:
            if os.path.isfile(ha):
                hafile = open(ha, "r")
                self.hadata.append(hafile.readlines())
                hafile.close()
            else:
                print('Error: HA output file:', ha,'does not exist.')
                return self

        return self
    
    def readmode(self):
        if not self.hadata:
            self.readhaout()
        
        countline = 0
        label_freq = 'MODES         EIGV          FREQUENCIES     IRREP  IR   INTENS    RAMAN'
        label_end = 0
        labels = []
        for ha in self.hadata:
            while countline < len(ha):
                if label in ha[countline]:
                    bg_mode = countline
                    label_end = 1
                elif label_end and ha[countline]:
                    ed_mode = countline
                    labels.append([bg_mode, ed_mode])
                    break
                
                countline += 1
            
        nmode_line = ed_mode - bg_mode
        modes = []
        n_ha = len(self.hadata)
        for i in range(nmode_line + 1):
            for ha in range(self.hadata):
                freq = 
                modes[i] = Mode(rank=i+1,npoint=n_ha, freq=[], vol=[],  eigvt=[])
